In [1]:
%pwd

'c:\\Users\\amrha\\Downloads\\IEEE_AI_ChatBot\\research'

In [55]:
import os
os.chdir('../')

In [56]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import CharacterTextSplitter

In [4]:
def load_pdf(data):
    loader = DirectoryLoader(data, glob="**/*.pdf", loader_cls=PyPDFLoader)
    documents=loader.load()
    return documents

In [5]:
extracted_data = load_pdf('data/')

In [6]:
len(extracted_data)

34

## preproccessing

In [7]:
from typing import List
from langchain.schema import Document
def filter_to_minimal_docs(docs:List[Document]) -> List[Document]:
    """Given a list of Documents objects, return a new list of Document objects containing only 'source' in metadata and the original page contant."""
    minimal_docs :List[Document] = []

    for doc in docs:
        src= doc.metadata.get('source')
        minimal_docs.append(Document(page_content=doc.page_content, metadata={'source': src}))
    return minimal_docs

In [8]:
minimal_docs = filter_to_minimal_docs(extracted_data)

In [9]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
# spitting the documents into smaller chunks
def text_splitter(minimal_docs):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    texts_chunk=text_splitter.split_documents(minimal_docs)
    return texts_chunk

In [10]:
texts_chunk = text_splitter(minimal_docs)
print(len(texts_chunk))

133


In [11]:
from langchain.embeddings import HuggingFaceEmbeddings

def download_embeddings():
    """
    Download and return the HuggingFace embeddings model.
    """
    model_name = "sentence-transformers/all-MiniLM-L6-v2"
    embeddings = HuggingFaceEmbeddings(
        model_name=model_name
    )
    return embeddings

embedding = download_embeddings()

C:\Users\amrha\AppData\Local\Temp\ipykernel_54608\30168171.py:8: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(


In [12]:
embedding

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [13]:
vector=embedding.embed_query("Hello world")
print(len(vector))

384


In [14]:
import os
print(os.getcwd())  # Show current working directory
print(os.path.isfile('.env'))  # Check if .env exists in this directory

c:\Users\amrha\Downloads\IEEE_AI_ChatBot
True


In [15]:
from dotenv import load_dotenv
import os
result = load_dotenv()  # take environment variables from .env.
print(f".env loaded: {result}")
print(f"Current working directory: {os.getcwd()}")
print(f".env exists: {os.path.isfile('.env')}")

.env loaded: True
Current working directory: c:\Users\amrha\Downloads\IEEE_AI_ChatBot
.env exists: True


In [16]:
from dotenv import load_dotenv
load_dotenv()

True

In [17]:
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

In [29]:
from pinecone import Pinecone

In [30]:
pinecone_api_key = PINECONE_API_KEY
pc=Pinecone(api_key=pinecone_api_key)

In [31]:
pc

In [35]:
from pinecone import ServerlessSpec
index_name = "ieee-ai-chat"  # Only lowercase, numbers, and hyphens
if index_name not in pc.list_indexes():
    pc.create_index(
        name=index_name,
        dimension=384,  # Dimension of the embedding model
        metric="cosine",  # Similarity metric
        spec=ServerlessSpec(cloud="aws", region="us-east-1")
    )

In [36]:
index=pc.Index(index_name)

In [37]:
from langchain_pinecone import PineconeVectorStore
docsearch = PineconeVectorStore.from_documents(documents=texts_chunk,embedding=embedding, index_name=index_name)

In [38]:
# load existing index

from langchain_pinecone import PineconeVectorStore
docsearch = PineconeVectorStore.from_existing_index(embedding=embedding, index_name=index_name)

In [25]:
# # ADD More Data to the existing index
# dswish=Document(page_content="Diabetes is a chronic condition that affects how your body turns food into energy. Most of the food you eat is broken down into sugar (also called glucose) and released into your bloodstream. When your blood sugar goes up, it signals your pancreas to release insulin. Insulin acts like a key to let the blood sugar into your body's cells for use as energy.",
#                  metadata={"source": "diabetes.txt"})
# docsearch.add_documents(documents=[dswish])

In [39]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [40]:
retriever_docs=retriever.get_relevant_documents("What is AI Committee?")

In [41]:
retriever_docs

[Document(id='5db1457b-db63-413c-a0bb-3f295b383e00', metadata={'source': 'data\\IEEE Beni Suef Student Branch.pdf'}, page_content="- **AI Committee:** \n \n        ### **Introduction** \n \n        **A Journey of Learning, Innovation, and Teamwork** \n \n        In today's fast-paced digital world, artificial intelligence skills are no longer optional; they are \nessential. That's why the IEEE Artificial Intelligence Committee became the perfect place to start a \njourney of growth. From learning the basics of machine learning and data handling, we moved toward"),
 Document(id='6aa8b330-95c1-4544-b9fa-9eff7698e21b', metadata={'source': 'data\\IEEE Beni Suef Student Branch.pdf'}, page_content="# IEEE Beni Suef Student Branch Website Articles \n \n- **Technical Committees** \n \n    The Technical Committees are the scientific engine of IEEE Beni-Suef. They focus on developing \nstudents' skills in **computer science, engineering, and space fields**, turning theoretical knowledge \ninto p

In [42]:
from dotenv import load_dotenv
load_dotenv()

True

In [ ]:
# from langchain_openai import ChatOpenAI
# chatModel = ChatOpenAI(model="gpt-4o")

In [43]:
import google.generativeai as genai
import os

# ضيف الـ API key
genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))

# اختار الموديل (فيه اختيارات زي gemini-1.5-flash أو gemini-1.5-pro)
model = genai.GenerativeModel("gemini-1.5-flash")

# جرب تسأل سؤال
response = model.generate_content("What is Acromegaly and gigantism?")
print(response.text)

Acromegaly and gigantism are both caused by the same underlying problem: **excess growth hormone (GH) in the body**.  The difference lies primarily in *when* the excess GH occurs:

* **Gigantism:** This condition develops *before* puberty, when the growth plates in the bones are still open.  The excess GH causes excessive growth in height, resulting in individuals who are abnormally tall.  Other features may include enlarged organs and other features related to excess GH.

* **Acromegaly:** This condition develops *after* puberty, when the growth plates have closed.  Because the bones can no longer lengthen, the excess GH instead causes an enlargement of the bones in the hands, feet, and face.  This leads to characteristic features like:
    * Enlarged hands and feet (with thickened skin)
    * Protruding jaw (prognathism)
    * Thickened facial features
    * Enlarged nose and lips
    * Deepened voice
    * Headaches
    * Joint pain
    * Sleep apnea
    * Organ enlargement (heart, 

In [44]:
import google.generativeai as genai
chatModel = genai.GenerativeModel("gemini-1.5-flash")

In [45]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.prompts import ChatPromptTemplate

In [ ]:
system_prompt = (
"You are \"IEEE_AI_ChatBot\", the official assistant for IEEE Beni-Suef Student Branch. Your goal is to help students and members with accurate, polite, and actionable answers about IEEE activities, committees, learning resources, events, technical guidance, and how to join or contribute."
"Knowledge base (use when answering):"
"Technical Committees: Astronomy, Power, Embedded Systems, GNC, Frontend, Backend, AI, Cybersecurity, Data Analysis. These committees focus on turning theoretical knowledge into practical projects across computer science, engineering, and space fields."
"Non-Technical Committees: Social Media, Graphic Design, HR, PR, Logistics, Entrepreneurship. These committees handle organization, creativity, communications, and event delivery."
"Behavior & tone:"
"Default language: English. Match the user's language if they explicitly use another language."
"Be concise, friendly, and professional. Start with a short direct answer for quick questions, then offer an expanded explanation or next steps if needed."
"If the user asks for technical help (code, configs, architecture), provide runnable examples, clearly state assumptions, and use `df` for dataset variables unless the user requests a different name."
"For requests about events, joining, or organization, provide checklists, roles, timelines, and templates (e.g., meeting agenda, onboarding checklist)."
"Clarification policy:"
"If the users question is ambiguous, ask **one** short clarifying question, then proceed."
"Fallback & web search:"
"If you cannot confidently answer from your knowledge, perform a web search for up-to-date information. After searching, present a concise summary and prepend it with:"
"\"I searched for this and found the following:\""
"Then list findings and include source links or citations when available."
"If web access is unavailable, say: \"I don't have web access right now; here's what I can suggest based on common practice,\" and provide practical alternatives (internal IEEE contacts, how to verify, or typical steps)."
"Safety & quality:"
"Never invent facts. If unsure, say \"I don't know\" and offer ways to verify."
"Avoid sensitive personal data and partisan or medical/legal advice."
"Prioritize clarity and actionable guidance (steps, templates, examples)."
    "\n\n"
    "{context}"
)


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [47]:
# !pip install langchain-google-genai

In [48]:
from langchain_google_genai import ChatGoogleGenerativeAI
chatModel = ChatGoogleGenerativeAI(model="gemini-1.5-flash", google_api_key=os.getenv("GOOGLE_API_KEY"))

In [49]:
question_answer_chain = create_stuff_documents_chain(chatModel, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [51]:
response = rag_chain.invoke({"input": "What is AI Committee?"})
print(response["answer"])

The IEEE Beni-Suef Student Branch AI Committee is a community for students interested in learning and applying artificial intelligence.  It provides opportunities to learn machine learning, from basic concepts to advanced projects, and contribute to innovative AI solutions.


In [52]:
test_questions = [
    {"question": "What is AI Committee?", "answer": "The IEEE Beni-Suef Student Branch AI Committee is a community for students interested in learning and applying artificial intelligence.  It provides opportunities to learn machine learning, from basic concepts to advanced projects, and contribute to innovative AI solutions."},
]

In [53]:
results = []
for item in test_questions:
    response = rag_chain.invoke({"input": item["question"]})
    results.append({"question": item["question"], "model_answer": response["answer"], "true_answer": item["answer"]})

In [54]:
def f1_score(pred, true):
    pred_tokens = set(pred.split())
    true_tokens = set(true.split())
    common = pred_tokens & true_tokens
    if not common:
        return 0
    precision = len(common) / len(pred_tokens)
    recall = len(common) / len(true_tokens)
    return 2 * (precision * recall) / (precision + recall)

scores = [f1_score(r["model_answer"], r["true_answer"]) for r in results]
average_f1 = sum(scores) / len(scores)
print("Average F1 Score:", average_f1)

Average F1 Score: 0.6666666666666667
